In [27]:
%pip install langchain-community pypdf dashscope

  Using cached dashscope-1.25.7-py3-none-any.whl.metadata (7.1 kB)
  Using cached pycparser-2.23-py3-none-any.whl.metadata (993 bytes)
Using cached dashscope-1.25.7-py3-none-any.whl (1.3 MB)
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ----------------------- ---------------- 2.1/3.5 MB 29.6 MB/s eta 0:00:01
   -------------------------- ------------- 2.4/3.5 MB 6.4 MB/s eta 0:00:01
   -------------------------------------- - 3.4/3.5 MB 5.3 MB/s eta 0:00:01
   ---------------------------------------- 3.5/3.5 MB 5.2 MB/s  0:00:00
Using cached pycparser-2.23-py3-none-any.whl (118 kB)

   ---------- ----------------------------- 1/4 [cffi]
   -------------------- ------------------- 2/4 [cryptography]
   ------------------------------ --------- 3/4 [dashscope]
   ------------------------------ --------- 3/4 [dashscope]
   ---------------------------------------- 4/4 [dashscope]

Note: you may need to restart the kernel to use updated packages.


In [3]:
# 从电脑环境变量中导入
import os
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
print("OPENAI_API_KEY:", OPENAI_API_KEY is not None)
print("LANGCHAIN_API_KEY:", LANGCHAIN_API_KEY is not None)

# 设置环境变量以启用LangChain的跟踪功能
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_PROJECT"] = "llm-langchain-learn"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

OPENAI_API_KEY: True
LANGCHAIN_API_KEY: True


# 文档和文档加载器
LangChain 实现了一个 Document 抽象，它旨在表示一个文本单元和相关的元数据。它有三个属性
page_content：表示内容的字符串；
metadata：包含任意元数据的字典；
id：（可选）文档的字符串标识符。
`metadata` 属性可以捕获有关文档来源、与其他文档的关系以及其他信息。请注意，单个 `Document` 对象通常表示一个较大文档的片段。
我们可以根据需要生成示例文档：

In [4]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

## 加载文档
让我们把一个PDF加载成一系列Document物品

In [18]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print("页数", len(docs))

页数 107


PyPDFLoader装一Document每个PDF页面的对象。对于每个项目，我们都能轻松访问：
页面的字符串内容;
包含文件名和页码的元数据。

In [20]:
print(f"{fix_encoding(docs[0].page_content[:200])}\n")
print(docs[0].metadata)

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
FO

{'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': './nke-10k-2023.pdf', 'total_pages': 107, 'page': 0, 'page_label': '1'}


## 拆分
进一步拆分 PDF 将有助于确保文档相关部分的含义不会被周围的文本“冲淡”。
我们可以为此目的使用文本分割器。这里我们将使用一个简单的文本分割器，它根据字符进行分区。我们将把文档分割成 1000 个字符的块，块之间有 200 个字符的重叠。重叠有助于减少将语句与其相关的重要上下文分离的可能性。我们使用 `RecursiveCharacterTextSplitter`，它将使用换行符等常见分隔符递归地分割文档，直到每个块达到适当的大小。这是通用文本用例推荐的文本分割器。

我们设置 `add_start_index=True`，以便每个分割的 Document 在初始 Document 中开始的字符索引作为元数据属性“start_index”保留。

In [21]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True,
)
all_splits = text_splitter.split_documents(docs)
print("分块后数量:", len(all_splits))

分块后数量: 516


# 嵌入
向量搜索是存储和搜索非结构化数据（如非结构化文本）的常用方法。其思想是存储与文本关联的数字向量。给定一个查询，我们可以将其嵌入为相同维度的向量，并使用向量相似度度量（如余弦相似度）来识别相关文本。

In [ ]:
from langchain_community.embeddings import DashScopeEmbeddings

# 使用通义千问的 text-embedding-v3 模型（DashScope）
# 这里和官网的示例稍有不同
embeddings = DashScopeEmbeddings(
    model="text-embedding-v3",
    dashscope_api_key=OPENAI_API_KEY  # 使用你的阿里云 API Key
)

In [29]:
# 生成两个分块的向量表示
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)
# 验证向量长度相同（同一模型生成的向量维度总是一致的）
assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 1024

[-0.05425441637635231, -0.016869427636265755, -0.06312180310487747, -0.0004044165834784508, -0.014613688923418522, -0.02983020432293415, 0.025027036666870117, 0.06109941750764847, 0.024365872144699097, 0.07712294161319733]


# 向量存储
LangChain VectorStore 对象包含用于向存储添加文本和 `Document` 对象，以及使用各种相似度指标查询它们的方法。它们通常用嵌入模型进行初始化，这些模型决定了文本数据如何转换为数字向量。
LangChain 包含一套与不同向量存储技术集成的集成。一些向量存储由提供商（例如，各种云提供商）托管，需要特定的凭据才能使用；一些（例如 Postgres）在可以本地运行或通过第三方运行的独立基础设施中运行；其他可以用于轻量级工作负载的内存中运行。让我们选择一个向量存储：

就是可以把向量存在数据库等

In [31]:
# 这里采用内存
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embedding = embeddings)

# 索引文档
ids = vector_store.add_documents(all_splits)

嵌入通常将文本表示为“密集”向量，使得含义相似的文本在**几何上接近**。这使我们只需传入一个问题即可检索相关信息，而无需了解文档中使用的任何特定关键词。
根据与字符串查询的相似度返回文档：

In [33]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)
print(results[0])

page_content='direct to consumer operations sell products through the following number of retail stores in the United States:
U.S. RETAIL STORES NUMBER
NIKE Brand factory stores 213 
NIKE Brand in-line stores (including employee-only stores) 74 
Converse stores (including factory stores) 82 
TOTAL 369 
In the United States, NIKE has eight significant distribution centers. Refer to Item 2. Properties for further information.
2023 FORM 10-K 2' metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': './nke-10k-2023.pdf', 'total_pages': 107, 'page': 4, 'page_label': '5', 'start_index': 3125}


In [34]:
# 异步版本的相似度搜索
results = await vector_store.asimilarity_search("When was Nike incorporated?")

print(results[0])

page_content='Table of Contents
PART I
ITEM 1. BUSINESS
GENERAL
NIKE, Inc. was incorporated in 1967 under the laws of the State of Oregon. As used in this Annual Report on Form 10-K (this "Annual Report"), the terms "we," "us," "our,"
"NIKE" and the "Company" refer to NIKE, Inc. and its predecessors, subsidiaries and affiliates, collectively, unless the context indicates otherwise.
Our principal business activity is the design, development and worldwide marketing and selling of athletic footwear, apparel, equipment, accessories and services. NIKE is
the largest seller of athletic footwear and apparel in the world. We sell our products through NIKE Direct operations, which are comprised of both NIKE-owned retail stores
and sales through our digital platforms (also referred to as "NIKE Brand Digital"), to retail accounts and to a mix of independent distributors, licensees and sales' metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'cr

In [35]:
# 返回分数
results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.7591022307649888

page_content='Table of Contents
FISCAL 2023 NIKE BRAND REVENUE HIGHLIGHTSThe following tables present NIKE Brand revenues disaggregated by reportable operating segment, distribution channel and major product line:
FISCAL 2023 COMPARED TO FISCAL 2022
• NIKE, Inc. Revenues were $51.2 billion in fiscal 2023, which increased 10% and 16% compared to fiscal 2022 on a reported and currency-neutral basis, respectively.
The increase was due to higher revenues in North America, Europe, Middle East & Africa ("EMEA"), APLA and Greater China, which contributed approximately 7, 6,
2 and 1 percentage points to NIKE, Inc. Revenues, respectively.
• NIKE Brand revenues, which represented over 90% of NIKE, Inc. Revenues, increased 10% and 16% on a reported and currency-neutral basis, respectively. This
increase was primarily due to higher revenues in Men's, the Jordan Brand, Women's and Kids' which grew 17%, 35%,11% and 10%, respectively, on a wholesale
equivalent basis.' metad

In [36]:
# 根据与嵌入式查询的相似度进行搜索，返回文档
embedding = embeddings.embed_query("How were Nike's margins impacted in 2023?")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])


page_content='Table of Contents
GROSS MARGIN
FISCAL 2023 COMPARED TO FISCAL 2022
For fiscal 2023, our consolidated gross profit increased 4% to $22,292 million compared to $21,479 million for fiscal 2022. Gross margin decreased 250 basis points to
43.5% for fiscal 2023 compared to 46.0% for fiscal 2022 due to the following:
*Wholesale equivalent
The decrease in gross margin for fiscal 2023 was primarily due to:
• Higher NIKE Brand product costs, on a wholesale equivalent basis, primarily due to higher input costs and elevated inbound freight and logistics costs as well as
product mix;
• Lower margin in our NIKE Direct business, driven by higher promotional activity to liquidate inventory in the current period compared to lower promotional activity in
the prior period resulting from lower available inventory supply;
• Unfavorable changes in net foreign currency exchange rates, including hedges; and
• Lower off-price margin, on a wholesale equivalent basis.
This was partially offset by:'

# 检索器
LangChain `VectorStore` 对象不属于 @[Runnable] 的子类。LangChain @[Retrievers] 是 Runnables，因此它们实现了一组标准方法（例如，同步和异步 `invoke` 和 `batch` 操作）。虽然我们可以从向量存储构建检索器，但检索器也可以与非向量存储的数据源（例如外部 API）进行接口。

我们可以在不继承 `Retriever` 的情况下自己创建一个简单的版本。如果我们选择要用于检索文档的方法，我们可以轻松创建一个可运行对象。下面我们将围绕 `similarity_search` 方法构建一个：

In [37]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import chain

# 
@chain
def retriever(query: str) -> List[Document]:
    results = vector_store.similarity_search(query)
    return results

retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ]
)

[[Document(id='735fa2aa-5e7a-49c4-9709-41ad85ca1877', metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': './nke-10k-2023.pdf', 'total_pages': 107, 'page': 4, 'page_label': '5', 'start_index': 3125}, page_content='direct to consumer operations sell products through the following number of retail stores in the United States:\nU.S. RETAIL STORES NUMBER\nNIKE Brand factory stores 213 \nNIKE Brand in-line stores (including employee-only stores) 74 \nConverse stores (including factory stores) 82 \nTOTAL 369 \nIn the United States, NIKE has eight significant distribution centers. Refer to Item 2. Properties for further informat